<a href="https://colab.research.google.com/github/harish-git63/Demo1/blob/main/HARISH_STUDENT_DATABASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify, make_response
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import io
from PIL import Image as PILImage
import sqlite3
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # Required for flash messages

# Configure upload folder
UPLOAD_FOLDER = 'static/uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def init_db():
    conn = sqlite3.connect('students.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS students
        (register_number TEXT PRIMARY KEY,
         name TEXT NOT NULL,
         age INTEGER,
         course TEXT,
         profile_image TEXT)
    ''')
    conn.commit()
    conn.close()

@app.route('/')
def index():
    conn = sqlite3.connect('students.db')
    c = conn.cursor()
    c.execute('SELECT * FROM students')
    students = c.fetchall()
    conn.close()
    return render_template('index.html', students=students)

@app.route('/student/<register_number>')
def student_details(register_number):
    conn = sqlite3.connect('students.db')
    c = conn.cursor()
    c.execute('SELECT * FROM students WHERE register_number = ?', (register_number,))
    student = c.fetchone()
    conn.close()

    if student is None:
        flash('Student not found!', 'error')
        return redirect(url_for('index'))

    return render_template('student_details.html', student=student)

@app.route('/generate_pdf/<register_number>')
def generate_pdf(register_number):
    # Get student data
    conn = sqlite3.connect('students.db')
    c = conn.cursor()
    c.execute('SELECT * FROM students WHERE register_number = ?', (register_number,))
    student = c.fetchone()
    conn.close()

    if student is None:
        flash('Student not found!', 'error')
        return redirect(url_for('index'))

    # Create PDF
    buffer = io.BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    elements = []

    # Title
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30
    )
    elements.append(Paragraph("Student Profile", title_style))
    elements.append(Spacer(1, 12))

    # Add profile image if exists
    if student[4]:
        try:
            img_path = os.path.join('static', student[4])
            if os.path.exists(img_path):
                img = PILImage.open(img_path)
                aspect = img.height / float(img.width)
                img_width = 2 * inch
                img_height = img_width * aspect
                elements.append(Image(img_path, width=img_width, height=img_height))
                elements.append(Spacer(1, 12))
        except Exception as e:
            print(f"Error processing image: {e}")

    # Student data
    data = [
        ["Register Number:", student[0]],
        ["Name:", student[1]],
        ["Age:", str(student[2]) if student[2] else "Not specified"],
        ["Course:", student[3] if student[3] else "Not specified"]
    ]

    table = Table(data, colWidths=[2*inch, 4*inch])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (0, -1), colors.grey),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 14),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 12),
        ('BACKGROUND', (1, 0), (-1, -1), colors.beige),
        ('TEXTCOLOR', (1, 0), (-1, -1), colors.black),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))

    elements.append(table)

    # Build PDF
    doc.build(elements)
    buffer.seek(0)

    # Create response
    response = make_response(buffer.getvalue())
    response.mimetype = 'application/pdf'
    response.headers['Content-Disposition'] = f'attachment; filename={student[0]}_profile.pdf'

    return response

@app.route('/get_student/<register_number>')
def get_student(register_number):
    conn = sqlite3.connect('students.db')
    c = conn.cursor()
    c.execute('SELECT * FROM students WHERE register_number = ?', (register_number,))
    student = c.fetchone()
    conn.close()

    if student is None:
        return jsonify({'error': 'Student not found'}), 404

    return jsonify({
        'register_number': student[0],
        'name': student[1],
        'age': student[2],
        'course': student[3],
        'profile_image': student[4]
    })

@app.route('/add_student', methods=['POST'])
def add_student():
    try:
        register_number = request.form['register_number']
        name = request.form['name']
        age = request.form['age']
        course = request.form['course']

        if not name or not register_number:
            flash('Name and Register Number are required!', 'error')
            return redirect(url_for('index'))

        # Handle file upload
        profile_image = request.files['profile_image']
        image_path = None

        if profile_image and allowed_file(profile_image.filename):
            filename = secure_filename(f"{register_number}_{profile_image.filename}")
            # Create uploads directory if it doesn't exist
            os.makedirs('static/uploads', exist_ok=True)
            # Save the file directly to the uploads folder
            profile_image.save(os.path.join('static/uploads', filename))
            # Store the relative path in the database
            image_path = f'uploads/{filename}'

        conn = sqlite3.connect('students.db')
        c = conn.cursor()

        # Check if this is an update operation
        is_update = request.form.get('is_update') == 'true'

        if is_update:
            old_register_number = request.form.get('old_register_number')
            if old_register_number != register_number:
                # Check if new register number exists
                c.execute('SELECT * FROM students WHERE register_number = ?', (register_number,))
                if c.fetchone() is not None:
                    flash('Register Number already exists!', 'error')
                    return redirect(url_for('index'))

            # Get old image path
            c.execute('SELECT profile_image FROM students WHERE register_number = ?', (old_register_number,))
            old_image = c.fetchone()

            # Update query
            if image_path:
                # Delete old image if exists and new image is uploaded
                if old_image and old_image[0]:
                    old_image_path = os.path.join('static', old_image[0])
                    if os.path.exists(old_image_path):
                        os.remove(old_image_path)
                c.execute('''
                    UPDATE students
                    SET register_number=?, name=?, age=?, course=?, profile_image=?
                    WHERE register_number=?
                ''', (register_number, name, age, course, image_path, old_register_number))
            else:
                c.execute('''
                    UPDATE students
                    SET register_number=?, name=?, age=?, course=?
                    WHERE register_number=?
                ''', (register_number, name, age, course, old_register_number))

            flash('Student updated successfully!', 'success')
        else:
            # Check for duplicate register number for new entries
            c.execute('SELECT * FROM students WHERE register_number = ?', (register_number,))
            if c.fetchone() is not None:
                flash('Register Number already exists!', 'error')
                return redirect(url_for('index'))

            c.execute('''
                INSERT INTO students (register_number, name, age, course, profile_image)
                VALUES (?, ?, ?, ?, ?)
            ''', (register_number, name, age, course, image_path))
            flash('Student added successfully!', 'success')

        conn.commit()
        conn.close()

    except Exception as e:
        flash(f'Error adding student: {str(e)}', 'error')

    return redirect(url_for('index'))

@app.route('/search', methods=['POST'])
def search():
    search_term = request.form['search_term']
    conn = sqlite3.connect('students.db')
    c = conn.cursor()
    c.execute('''
        SELECT * FROM students
        WHERE register_number LIKE ? OR name LIKE ?
    ''', (f'%{search_term}%', f'%{search_term}%'))
    students = c.fetchall()
    conn.close()
    return render_template('index.html', students=students, search_term=search_term)

@app.route('/delete/<register_number>')
def delete_student(register_number):
    try:
        conn = sqlite3.connect('students.db')
        c = conn.cursor()

        # Get the image path before deleting the record
        c.execute('SELECT profile_image FROM students WHERE register_number = ?', (register_number,))
        result = c.fetchone()

        if result and result[0]:
            # Delete the image file if it exists
            image_path = os.path.join('static', result[0])
            if os.path.exists(image_path):
                os.remove(image_path)

        # Delete the database record
        c.execute('DELETE FROM students WHERE register_number = ?', (register_number,))
        conn.commit()
        conn.close()

        flash('Student deleted successfully!', 'success')
    except Exception as e:
        flash(f'Error deleting student: {str(e)}', 'error')

    return redirect(url_for('index'))

if __name__ == '__main__':
    init_db()
    app.run(debug=True)


TclError: no display name and no $DISPLAY environment variable